In [200]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Import the data set

In [201]:
lab_data = pd.read_csv('../../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data Cleaning

In [202]:
lab_data = pd.read_csv('../../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data cleaning
erroneous = lab_data[lab_data['Thorax_length'] == '.']
erroneous

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
253,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,25,6,3,female,.,1.65,0.573,1.415,1.988,1.986,0.898,1.207,1.003,.


In [203]:
similar_rows = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Gogango_Creek')
    & (lab_data['Sex'] == 'female')
    & (lab_data['Thorax_length'] != ".")
]
similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])
similar_rows.head()

/var/folders/j_/9f1md2vd0_s6kpn12szf1t2c0000gn/T/ipykernel_2918/3776846164.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])


,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
162,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,1,female,1.166,1.910,0.578,1.675,2.254,2.252,0.982,1.381,1.123,1.932
164,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,2,female,1.174,1.829,0.622,1.549,2.170,2.170,0.997,1.360,1.117,1.848
166,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,3,female,1.199,1.864,0.585,1.659,2.244,2.242,1.009,1.376,1.142,1.870
167,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,1,female,1.186,1.872,0.628,1.571,2.199,2.199,1.002,1.356,1.143,1.854
169,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,2,female,1.176,1.872,0.630,1.579,2.207,2.207,0.966,1.345,1.103,1.876


In [204]:
similar_row_median = similar_rows['Thorax_length'].median()

lab_data['Thorax_length'] = lab_data.apply(
    lambda x: (x['Thorax_length'] if x['Thorax_length'] != "." else similar_row_median),
    axis=1
)
lab_data.head()

lab_data['wing_loading'] = lab_data.apply(
    lambda x: (x['wing_loading'] if x['wing_loading'] != "." else round((x['l3'] / x['Thorax_length']), 3)),
    axis=1
)

print(len(similar_rows))
print(similar_row_median)

lab_data.loc[253]

87
1.141


Species            D._aldrichi
Population       Gogango_Creek
Latitude                -23.77
Longitude               150.17
Year_start                1994
Year_end                  1994
Temperature                 25
Vial                         6
Replicate                    3
Sex                     female
Thorax_length            1.141
l2                        1.65
l3p                      0.573
l3d                      1.415
lpd                      1.988
l3                       1.986
w1                       0.898
w2                       1.207
w3                       1.003
wing_loading             1.741
Name: 253, dtype: object

In [205]:
similar_row_median = similar_rows['Thorax_length'].median()

lab_data['Thorax_length'] = lab_data.apply(
    lambda x: (x['Thorax_length'] if x['Thorax_length'] != "." else similar_row_median),
    axis=1
)
lab_data.head()

lab_data['wing_loading'] = lab_data.apply(
    lambda x: (x['wing_loading'] if x['wing_loading'] != "." else round((x['l3'] / x['Thorax_length']), 3)),
    axis=1
)

print(len(similar_rows))
print(similar_row_median)

lab_data.loc[253]

87
1.141


Species            D._aldrichi
Population       Gogango_Creek
Latitude                -23.77
Longitude               150.17
Year_start                1994
Year_end                  1994
Temperature                 25
Vial                         6
Replicate                    3
Sex                     female
Thorax_length            1.141
l2                        1.65
l3p                      0.573
l3d                      1.415
lpd                      1.988
l3                       1.986
w1                       0.898
w2                       1.207
w3                       1.003
wing_loading             1.741
Name: 253, dtype: object

In [206]:
lab_data['Thorax_length'] = pd.to_numeric(lab_data['Thorax_length'])
lab_data['wing_loading'] = pd.to_numeric(lab_data['wing_loading'])

In [207]:
erroneous_2 = lab_data[(lab_data['l3d'] == 0) & (lab_data['l3p'] != 0)]
erroneous_2

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
61,D._aldrichi,Binjour,-25.52,151.45,1994,1994,25,3,1,female,1.106,0.0,0.6,0.0,0.0,0.0,0.0,1.252,0.0,0.0


In [208]:
similar_rows_2 = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Binjour')
    & (lab_data['Sex'] == 'female')
    & (lab_data['l3d'] != "0")
]

similar_rows_2_l2_median = similar_rows_2['l2'].median()
similar_rows_2_l3d_median = similar_rows_2['l3d'].median()
similar_rows_2_lpd_median = similar_rows_2['lpd'].median()
similar_rows_2_l3_median = similar_rows_2['l3'].median()
similar_rows_2_w1_median = similar_rows_2['w1'].median()
similar_rows_2_w3_median = similar_rows_2['w3'].median()
lab_data.loc[61, 'l2'] = similar_rows_2_l2_median
lab_data.loc[61, 'l3d'] = similar_rows_2_l3d_median
lab_data.loc[61, 'lpd'] = similar_rows_2_lpd_median
lab_data.loc[61, 'l3'] = similar_rows_2_l3_median
lab_data.loc[61, 'w1'] = similar_rows_2_w1_median
lab_data.loc[61, 'w3'] = similar_rows_2_w3_median
lab_data.loc[61, 'wing_loading'] = lab_data.loc[61]['l3'] / lab_data.loc[61]['Thorax_length']
lab_data.loc[[61]]

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
61,D._aldrichi,Binjour,-25.52,151.45,1994,1994,25,3,1,female,1.106,1.799,0.6,1.5235,2.1195,2.1185,0.9235,1.252,1.0495,1.915461


In [209]:
erroneous_3 = lab_data[(lab_data['l3d'] == 0) & (lab_data['l3p'] == 0)]
erroneous_3

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
698,D._aldrichi,Wahruna,-25.2,151.17,1994,1994,20,5,3,female,1.151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [210]:
#TODO: 
similar_rows_3 = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Wahruna')
    & (lab_data['Sex'] == 'female')
    & (lab_data['l3d'] != "0")
]
len(similar_rows_3)

84

In [211]:
similar_rows_3_l2_median = similar_rows_3['l2'].median()
similar_rows_3_l3p_median = similar_rows_3['l3p'].median()
similar_rows_3_l3d_median = similar_rows_3['l3d'].median()
similar_rows_3_lpd_median = similar_rows_3['lpd'].median()
similar_rows_3_l3_median = similar_rows_3['l3'].median()
similar_rows_3_w1_median = similar_rows_3['w1'].median()
similar_rows_3_w2_median = similar_rows_3['w2'].median()
similar_rows_3_w3_median = similar_rows_3['w3'].median()


lab_data.loc[698, 'l2'] = similar_rows_3_l2_median
lab_data.loc[698, 'l3p'] = similar_rows_3_l3p_median
lab_data.loc[698, 'l3d'] = similar_rows_3_l3d_median
lab_data.loc[698, 'lpd'] = similar_rows_3_lpd_median
lab_data.loc[698, 'l3'] = similar_rows_3_l3_median
lab_data.loc[698, 'w1'] = similar_rows_3_w1_median
lab_data.loc[698, 'w2'] = similar_rows_3_w2_median
lab_data.loc[698, 'w3'] = similar_rows_3_w3_median
lab_data.loc[698, 'wing_loading'] = lab_data.loc[698]['l3'] / lab_data.loc[698]['Thorax_length']

lab_data.loc[[698]]

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
698,D._aldrichi,Wahruna,-25.2,151.17,1994,1994,20,5,3,female,1.151,1.7405,0.6,1.4925,2.0855,2.084,0.943,1.2805,1.0645,1.810599


# Model Building

In [212]:
lab_data['Thorax_length'] = pd.to_numeric(lab_data['Thorax_length'])
lab_data['wing_loading'] = pd.to_numeric(lab_data['wing_loading'])

In [213]:
lab_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1731 entries, 0 to 1730
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Species        1731 non-null   object 
 1   Population     1731 non-null   object 
 2   Latitude       1731 non-null   float64
 3   Longitude      1731 non-null   float64
 4   Year_start     1731 non-null   int64  
 5   Year_end       1731 non-null   int64  
 6   Temperature    1731 non-null   int64  
 7   Vial           1731 non-null   int64  
 8   Replicate      1731 non-null   int64  
 9   Sex            1731 non-null   object 
 10  Thorax_length  1731 non-null   float64
 11  l2             1731 non-null   float64
 12  l3p            1731 non-null   float64
 13  l3d            1731 non-null   float64
 14  lpd            1731 non-null   float64
 15  l3             1731 non-null   float64
 16  w1             1731 non-null   float64
 17  w2             1731 non-null   float64
 18  w3      

In [214]:
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
0,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,female,1.238,2.017,0.659,1.711,2.370,2.370,1.032,1.441,1.192,1.914
1,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,male,1.113,1.811,0.609,1.539,2.148,2.146,0.938,1.299,1.066,1.928
2,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,female,1.215,1.985,0.648,1.671,2.319,2.319,0.991,1.396,1.142,1.908
3,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,male,1.123,1.713,0.596,1.495,2.091,2.088,0.958,1.286,1.062,1.860
4,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,2,1,female,1.218,1.938,0.641,1.658,2.298,2.298,1.010,1.418,1.148,1.886


In [215]:
#TODO: Currently using decision tree so no need to normalise, but must do for other methods

In [216]:
lab_data = shuffle(lab_data)
lab_data['class'] = lab_data.apply(
    lambda x: x['Sex'],
    axis=1
)
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class
360,D._aldrichi,Grandchester,-27.68,152.45,1994,1994,20,4,3,male,...,1.820,0.625,1.558,2.183,2.183,1.014,1.324,1.172,1.902,male
1065,D._buzzatii,Gogango_Creek,-23.77,150.17,1994,1994,20,8,3,female,...,1.994,0.666,1.648,2.314,2.313,1.041,1.432,1.203,1.880,female
146,D._aldrichi,Binjour,-25.52,151.45,1994,1994,30,8,1,male,...,1.506,0.481,1.319,1.800,1.798,0.804,1.113,0.900,1.835,male
114,D._aldrichi,Binjour,-25.52,151.45,1994,1994,30,2,2,male,...,1.537,0.509,1.347,1.856,1.854,0.846,1.162,0.928,1.777,male
30,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,7,2,female,...,1.975,0.666,1.640,2.306,2.305,1.031,1.439,1.166,1.901,female


In [217]:

train, test = train_test_split(lab_data, test_size=0.2)

In [218]:
classifier = DecisionTreeClassifier(criterion='entropy', max_depth=5)
#TODO: Add Thorax Length and wing_loading to this training
classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [219]:
train['class_predictions'] = classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [220]:
train.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class,class_predictions
111,D._aldrichi,Binjour,-25.52,151.45,1994,1994,30,2,1,female,...,0.565,1.440,2.004,2.003,0.891,1.234,1.011,1.833,female,male
1729,D._buzzatii,Wahruna,-25.20,151.17,1994,1994,30,10,3,female,...,0.544,1.404,1.947,1.947,0.863,1.187,0.976,1.741,female,female
310,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,30,6,1,male,...,0.465,1.233,1.698,1.698,0.783,1.042,0.870,1.767,male,male
1590,D._buzzatii,Wahruna,-25.20,151.17,1994,1994,20,7,1,male,...,0.642,1.518,2.160,2.160,0.946,1.322,1.088,1.871,male,male
371,D._aldrichi,Grandchester,-27.68,152.45,1994,1994,20,6,3,female,...,0.609,1.651,2.261,2.260,1.043,1.381,1.198,1.844,female,female


In [221]:
#Correct predictions
train.apply(
    lambda x: 1 if x['class_predictions'] == x['class'] else 0,
    axis=1
).sum()

1186

In [222]:
#Incorrect predictions
train.apply(
    lambda x: 1 if x['class_predictions'] != x['class'] else 0,
    axis=1
).sum()

198

In [223]:
# Retry with a deeper depth

In [224]:
deep_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=10)

deep_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [225]:
train['deep_class_predictions'] = deep_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [226]:
#Correct deep predictions
train.apply(
    lambda x: 1 if x['deep_class_predictions'] == x['class'] else 0,
    axis=1
).sum()

1291

In [227]:
min_leaf_classifier = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

min_leaf_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

In [228]:
train['min_leaf_predictions'] = min_leaf_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [229]:
#Correct min_leaf predictions
train.apply(
    lambda x: 1 if x['min_leaf_predictions'] == x['class'] else 0,
    axis=1
).sum()

1306

In [230]:
fully_fit_classifier = DecisionTreeClassifier(criterion='entropy')

fully_fit_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy')

In [231]:
train['fully_fit_predictions'] = fully_fit_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [232]:
#Correct fully_fit predictions
train.apply(
    lambda x: 1 if x['fully_fit_predictions'] == x['class'] else 0,
    axis=1
).sum()

1384

In [233]:
# Checking the min leaf and fully fit classifiers against the test set

In [234]:
#Correst min_leaf predictions on the test set

In [235]:
test['class_predictions'] = classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['deep_class_predictions'] = deep_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['min_leaf_predictions'] = min_leaf_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['fully_fit_predictions'] = fully_fit_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

correct_classification_test = test.apply(
    lambda x: 1 if x['class_predictions'] == x['class'] else 0,
    axis=1
).sum()


correct_deep_classification_test = test.apply(
    lambda x: 1 if x['deep_class_predictions'] == x['class'] else 0,
    axis=1
).sum()

correct_min_leaf_classification_test = test.apply(
    lambda x: 1 if x['min_leaf_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Min leaf correct predictions test set:" + str(correct_min_leaf_classification_test))

correct_fully_fit_classification_test = test.apply(
    lambda x: 1 if x['fully_fit_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Fully fit predictions test set:" + str(correct_fully_fit_classification_test))


print(f"accuracy: {correct_classification_test / len(test)}")

Min leaf correct predictions test set:266
Fully fit predictions test set:266
accuracy: 0.829971181556196


In [197]:
print(f"accuracy: {correct_classification_test / len(test)}")

accuracy: 0.8097982708933718


In [198]:
print(f"accuracy: {correct_deep_classification_test / len(test)}")

accuracy: 0.792507204610951


In [199]:
print(f"accuracy: {correct_fully_fit_classification_test / len(test)}")

accuracy: 0.7406340057636888


In [121]:
print(f"accuracy: {correct_min_leaf_classification_test / len(test)}")

accuracy: 0.7636887608069164
